# Webscrapping festivos España
Para recopliar los festivos de España desde 2015 hasta 2025.

Definiendo las provincias, CCAA, meses literales, etc.

In [ ]:
provincia_ccaa = {
    "Alava": "Pais Vasco",
    "Albacete": "Castilla-La Mancha",
    "Alicante": "Comunidad Valenciana",
    "Almeria": "Andalucia",
    "Asturias": "Principado de Asturias",
    "Avila": "Castilla y Leon",
    "Badajoz": "Extremadura",
    "Barcelona": "Cataluna",
    "Burgos": "Castilla y Leon",
    "Caceres": "Extremadura",
    "Cadiz": "Andalucia",
    "Cantabria": "Cantabria",
    "Castellon": "Comunidad Valenciana",
    "Ciudad Real": "Castilla-La Mancha",
    "Cordoba": "Andalucia",
    "La Coruna": "Galicia",
    "Cuenca": "Castilla-La Mancha",
    "Gerona": "Cataluna",
    "Granada": "Andalucia",
    "Guadalajara": "Castilla-La Mancha",
    "Guipuzcoa": "Pais Vasco",
    "Huelva": "Andalucia",
    "Huesca": "Aragon",
    "Islas Baleares": "Illes Balears",
    "Jaen": "Andalucia",
    "Leon": "Castilla y Leon",
    "Lerida": "Cataluna",
    "Lugo": "Galicia",
    "Madrid": "Comunidad de Madrid",
    "Malaga": "Andalucia",
    "Murcia": "Region de Murcia",
    "Navarra": "Comunidad Foral de Navarra",
    "Orense": "Galicia",
    "Palencia": "Castilla y Leon",
    "Las Palmas": "Canarias",
    "Pontevedra": "Galicia",
    "La Rioja": "La Rioja",
    "Salamanca": "Castilla y Leon",
    "Santa Cruz de Tenerife": "Canarias",
    "Segovia": "Castilla y Leon",
    "Sevilla": "Andalucia",
    "Soria": "Castilla y Leon",
    "Tarragona": "Cataluna",
    "Teruel": "Aragon",
    "Toledo": "Castilla-La Mancha",
    "Valencia": "Comunidad Valenciana",
    "Valladolid": "Castilla y Leon",
    "Vizcaya": "Pais Vasco",
    "Zamora": "Castilla y Leon",
    "Zaragoza": "Aragon",
    "Ceuta": "Ceuta",
    "Melilla": "Melilla"
}


provincias = [
    "Alava", "Albacete", "Alicante", "Almeria", "Asturias", "Avila",
    "Badajoz", "Barcelona", "Burgos", "Caceres", "Cadiz", "Cantabria",
    "Castellon", "Ciudad Real", "Cordoba", "La Coruna", "Cuenca", "Gerona",
    "Granada", "Guadalajara", "Guipuzcoa", "Huelva", "Huesca", "Islas Baleares",
    "Jaen", "Leon", "Lerida", "Lugo", "Madrid", "Malaga", "Murcia", "Navarra",
    "Orense", "Palencia", "Las Palmas", "Pontevedra", "La Rioja", "Salamanca",
    "Santa Cruz de Tenerife", "Segovia", "Sevilla", "Soria", "Tarragona",
    "Teruel", "Toledo", "Valencia", "Valladolid", "Vizcaya", "Zamora", "Zaragoza"
]

# provincias = [
#     "Alava", "Albacete"
# ]

meses = [
    "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

def get_ccaa(provincia):
    """Devuelve la comunidad autónoma de una provincia española."""
    return provincia_ccaa.get(provincia, "Provincia no encontrada")

Importando las principales librerias:

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

Función de scrapeo de la página [https://www.calendarioslaborales.com/calendario-laboral](https://www.calendarioslaborales.com/calendario-laboral)

In [11]:
def webscrap_calendar(provincia, anio):
    # provincia = "Madrid"
    # anio = 2025
    # comunidad = "Madrid"
    festivos_loc = []
    url = f"https://www.calendarioslaborales.com/calendario-laboral-{provincia.lower()}-{anio}.htm"
    print(url)
    res = requests.get(url)
    soup = bs(res.text, "html.parser")


    for div in soup.find_all("div", class_="wrapFestivos"):
        for li in div.find_all("li"):
            clase = li.find("span").get("class", [""])[0] 
            ambito = {
                "festivoN": "nacional",
                "festivoP": "local",
                "festivoR": "regional"
            }.get(clase, "desconocido")
            
            span = li.find("span")
            fecha = span.text.strip().rstrip('.').split()
            festivo_nombre = li.get_text().replace(span.text, "").strip()
            
            festivos_loc.append({
                "dia": int(fecha[0]),
                "mes": fecha[-1],
                "anio": anio,
                "festivo": festivo_nombre,
                "provincia": provincia,
                "comunidad": get_ccaa(provincia),
                "ambito": ambito,
            })

    df_festivos = pd.DataFrame(festivos_loc)
    return df_festivos


Llamada principal para todas las provincias:

In [ ]:
dfs = []
for provincia in provincias:
    for anio in range(2015, 2026):
        df_fes = webscrap_calendar(provincia, anio)
        dfs.append(df_fes)

In [13]:
df_final= pd.concat(dfs, ignore_index=True)
df_final.head()

,dia,mes,anio,festivo,provincia,comunidad,ambito
0,1,Enero,2015,Año nuevo,Alava,Pais Vasco,nacional
1,6,Enero,2015,Epifanía del Señor,Alava,Pais Vasco,nacional
2,19,Marzo,2015,San José,Alava,Pais Vasco,regional
3,2,Abril,2015,Jueves Santo,Alava,Pais Vasco,regional
4,3,Abril,2015,Viernes Santo,Alava,Pais Vasco,nacional


In [14]:
df_final.tail()

,dia,mes,anio,festivo,provincia,comunidad,ambito
6274,12,Octubre,2025,Fiesta Nacional(se traslada al lunes),Zaragoza,Aragon,regional
6275,1,Noviembre,2025,Día de Todos los Santos,Zaragoza,Aragon,nacional
6276,6,Diciembre,2025,Día de la Constitución Española,Zaragoza,Aragon,nacional
6277,8,Diciembre,2025,Inmaculada Concepción,Zaragoza,Aragon,nacional
6278,25,Diciembre,2025,Navidad,Zaragoza,Aragon,nacional


In [15]:
df_final.to_csv('./data/calendario_festivos_15_25.csv')